In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# DN = 'C://work/dev/python/progs/texts/sec_bert/'
DN = '/home/jovyan/work/sec_bert/'

In [4]:
import os
os.chdir(DN)

In [5]:
import pandas as pd
import numpy as np
import joblib
import json
from itertools import chain
import click

from ruamel.yaml import YAML

In [7]:
conf = YAML().load(open('params.yaml'))
mitre_attack_df = pd.read_csv(conf['get_data']['data_mitre_attack_proc_fn'])
mitre_attack_df = mitre_attack_df.assign(labels = mitre_attack_df['labels'].map(lambda x:[x]))

# mitre_attack_df = mitre_attack_df[mitre_attack_df['sentence'].str.split().str.len()>=10].reset_index(drop=True)
# write same sentences for different technick 
mitre_attack_df[(mitre_attack_df.duplicated('sentence', keep=False))
        &(mitre_attack_df['sentence']!=' ')\
        &(mitre_attack_df['sentence'].str.split().str.len()>=10)\
        &(~mitre_attack_df['sentence'].str.contains('*', regex=False))].sort_values(by='sentence')\
        .to_csv('data/artifacts/duple_text_dif_techniks.csv')

# mitre_attack_df = mitre_attack_df.drop_duplicates('sentence')

with open(conf['get_data']['label2tactic_fn'], 'rt') as f_r:
    label2tactic = json.load(f_r)

tram_df = pd.read_json(conf['get_data']['tram_fn']).drop(columns='doc_title')
sel = tram_df.sentence.str.findall(';').str.len()>0
tram_df[sel].to_csv('data/artifacts/tram_rubbish.csv', index=False)
tram_df = tram_df[~sel]
tram_df = tram_df.drop_duplicates(subset='sentence')




if conf['get_data']['use_alt_mitre']:
    par_d = mitre_attack_df.explode('labels').set_index('labels')['par_name'].to_dict()
    mitr_alt_data = pd.read_csv(conf['get_data']['mitre_alt_fn'])
    mitr_alt_data['labels'] = mitr_alt_data[['label_subtec', 'label_tec']].apply(lambda x: list(set((x[0], x[1]))), axis=1)
    mitr_alt_data = mitr_alt_data[['sentence', 'labels']]
    mitr_alt_data['par_name'] = mitr_alt_data['labels'].map(lambda x: sorted([par_d[it] if it in par_d and len(it)>6 else '' for it in x])[-1])

    mitr_df = mitr_alt_data

else:
    mitr_df = mitre_attack_df

if conf['get_data']['use_tram_f']:
    df = pd.concat([mitr_df, tram_df], ignore_index=True)
else:
    df = mitr_df
    
if conf['get_data']['use_reports_f']:
    DN = conf['get_data']['rep_dn']
    fns = [f'{DN}/{it}' for it in os.listdir(DN) if 'json' in it]
    
    tab_df = pd.concat([pd.read_json(fn).explode('tables') for fn in fns], ignore_index=True)
    tab_df['tables'] = tab_df['tables'].map(lambda x: [(it1, it2) 
                                    for it1, it2 in zip(x['TechniqueID'].values(), x['Procedure'].values())])
    tab_df = tab_df.explode('tables').drop_duplicates().reset_index(drop=True)
    tab_df['technic'] = tab_df['tables'].map(lambda x: x[0])
    tab_df['sentence'] = tab_df['tables'].map(lambda x: x[1])
    
    tab_df = tab_df[tab_df['sentence'].str.split().str.len()>3]
    tab_df['labels'] = tab_df['technic'].map(lambda x: [x.upper()])
    
    # тут и мобильные угрозы есть, поэтому фильтруем их
    tab_df = tab_df[tab_df['labels'].map(lambda x: all([it in label2tactic for it in x]))]

    df = pd.concat([df, tab_df[['sentence',	'labels', 'report_path']].rename(columns={'report_path':'url'})], ignore_index=True)

df['origin_labels'] = df['labels']

In [ ]:
дана строка с точкой

In [12]:
df

/opt/conda/lib/python3.11/site-packages/IPython/lib/pretty.py:787: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/opt/conda/lib/python3.11/site-packages/IPython/core/formatters.py:347: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,sentence,labels,url,par_name,is_proc,origin_labels,origin_labels2
0,Adversaries may inject malicious code into pro...,[T1055.011],https://attack.mitre.org/techniques/T1055/011,Process Injection,False,[T1055.011],[T1055]
1,"Before creating a window, graphical Windows-ba...",[T1055.011],https://attack.mitre.org/techniques/T1055/011,Process Injection,False,[T1055.011],[T1055]
2,"Although small, the EWM is large enough to sto...",[T1055.011],https://attack.mitre.org/techniques/T1055/011,Process Injection,False,[T1055.011],[T1055]
3,Execution granted through EWM injection may al...,[T1055.011],https://attack.mitre.org/techniques/T1055/011,Process Injection,False,[T1055.011],[T1055]
4,Running code in the context of another process...,[T1055.011],https://attack.mitre.org/techniques/T1055/011,Process Injection,False,[T1055.011],[T1055]
...,...,...,...,...,...,...,...
35373,Requires the victim to run the malware .out file.,[T1204.002],/opt/nifi/configuration_resources/rstthreat/ti...,NaN,NaN,[T1204.002],[T1204]
35374,Portions of files are encoded to hide the plai...,[T1027],/opt/nifi/configuration_resources/rstthreat/ti...,NaN,NaN,[T1027],[T1027]
35375,Uses macros to execute payloads,[T1059.005],/opt/nifi/configuration_resources/rstthreat/ti...,NaN,NaN,[T1059.005],[T1059]
35376,Uses HTTP GET requests to contact the command-...,[T1071.001],/opt/nifi/configuration_resources/rstthreat/ti...,NaN,NaN,[T1071.001],[T1071]
